In [49]:
import pandas as pd
import numpy as np

from datetime import datetime
import time

from collections import defaultdict

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


%matplotlib inline

from math import log
from collections import Counter
import re



import os, sys, email
import numpy as np 
import pandas as pd
# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')
#import plotly
#plotly.offline.init_notebook_mode()
#import plotly.graph_objs as go
import wordcloud

# Network analysis
import networkx as nx
# NLP
from nltk.tokenize.regexp import RegexpTokenizer

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))



ModuleNotFoundError: No module named 'wordcloud'

In [47]:
emails_df = pd.read_csv('emails.csv', nrows=100000)
emails_df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [8]:
s = df.message[24631]
#s = s.replace('\n', ' ')
s

"Message-ID: <20249642.1075855765219.JavaMail.evans@thyme>\nDate: Wed, 22 Nov 2000 23:31:00 -0800 (PST)\nFrom: bryan.seyfried@enron.com\nTo: richard.causey@enron.com, rick.buy@enron.com, john.sherriff@enron.com\nSubject: Debt Trading Losses\nCc: michael.brown@enron.com, fernley.dyson@enron.com, sally.beck@enron.com, \n\tted.murphy@enron.com, mike.jordan@enron.com, david.wall@enron.com, \n\tjeff.kinneman@enron.com, gary.hickerson@enron.com, \n\tsheila.glover@enron.com, brent.price@enron.com, \n\trichard.sage@enron.com, bryan.seyfried@enron.com\nMime-Version: 1.0\nContent-Type: text/plain; charset=ANSI_X3.4-1968\nContent-Transfer-Encoding: 7bit\nBcc: michael.brown@enron.com, fernley.dyson@enron.com, sally.beck@enron.com, \n\tted.murphy@enron.com, mike.jordan@enron.com, david.wall@enron.com, \n\tjeff.kinneman@enron.com, gary.hickerson@enron.com, \n\tsheila.glover@enron.com, brent.price@enron.com, \n\trichard.sage@enron.com, bryan.seyfried@enron.com\nX-From: Bryan Seyfried\nX-To: Richard C

In [5]:
dict_month = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}
def pars(message):

    date = re.findall(r'\d{1,2}\s\w+\s\d{4}\s\d{2}:\d{2}:\d{2}', message)[0]
    month = re.findall(r'\s(\D+)\s', date)[0]
    date = date.replace(month, dict_month[month])
    date_list = date.split(' ')
    date_ = '.'.join(date_list[:3])
    time = date_list[-1]
    date = ' '.join([date_, time])
    
    sender = re.findall(r'From:\s(.{,100}\@.{,100})', message)[0]
    recipients = re.findall(r'To:\s(.{,100000})', message)[0]
    subject = re.findall(r'Subject:\s(.{,100000})', message)[0]
    
    if re.findall(r'Cc:\s(.{,100000})', message):
        copy_rec = re.findall(r'Cc:\s(.{,100000})', message)[0]
    else: copy_rec = ' '
    
    message = message.replace('\n\n', ';;')
    message = message.replace('\n', ' ')
    text = re.findall(r'X-FileName:.{,100};;(.{,1000000})', message)[0]

    return (date, sender, recipients, copy_rec, subject, text)

pars(s)

('22.11.2000 23:31:00',
 'bryan.seyfried@enron.com',
 'richard.causey@enron.com, rick.buy@enron.com, john.sherriff@enron.com',
 'michael.brown@enron.com, fernley.dyson@enron.com, sally.beck@enron.com, ',
 'Debt Trading Losses',
 "We have discovered two positions in the Debt Trading book which were not  being marked to market. The current mark of the 2 positions results in  approximately $4.7 million loss.  The details are in :;;;;The primary control issues are: ? Trader did not adjust price to reflect the effect of market movements ? Operations did not perform monthly price verification processes ? Desk management failed to recognize that the marks on these two trades had  not moved over the term of the transactions (Q1 '00 - Nov 15th);;The losses have been recognized and a full review of the book is ongoing by  Operations.  The trader has been relieved of his responsibilities and we will  implement the steps agreed with Operations to ensure that we have a robust  approach to controls 

In [9]:
import datetime as dt
df1 = pd.DataFrame()
#df1['Date'] = df.message.apply(lambda x: pars(x)[0])
df1['Sender'] = df.message.apply(lambda x: pars(x)[1])
df1['Recipient'] = df.message.apply(lambda x: pars(x)[2])
df1['CopyRecipient'] = df.message.apply(lambda x: pars(x)[3])
df1['Subject'] = df.message.apply(lambda x: pars(x)[4])
df1['Text'] = df.message.apply(lambda x: pars(x)[5])

In [35]:
df1['Date'] = df.message.apply(lambda x: pars(x)[0])

In [41]:
df1['Date'][0]

'14.05.2001 16:39:00'

In [44]:
date = '14.05.0001 16:39:00'
date_list = date.split(' ')
date_1 = date_list[0]
date_1_list = date_1.split('.')
if date_1_list[2][0] == 0:
    date_1_list[2][0] = date_1_list[2][0].replace('0', '2')
date_1_list

['14', '05', '0001']

In [ ]:
def change_time(time):
    

In [45]:
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [50]:
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
emails_df['content'] = list(map(get_text_from_email, messages))
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages
emails_df.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p
